In [18]:
import pandas as pd

paper_stats_publication_classified = pd.read_csv("data/0510-citation-pub-data-publication-classified.csv")
paper_stats_speaker_order = pd.read_csv("data/0510-citation-pub-data-speaker-order-absolute.csv")

author_stats = pd.read_csv("data/0721-month-analysis-data-merged-5.csv")

/var/folders/rw/s7c9z30x7fj_2gcngv0yhv6w0000gn/T/ipykernel_13307/3257668968.py:3: DtypeWarning: Columns (12,21,22,24,33,34) have mixed types. Specify dtype option on import or set low_memory=False.
  paper_stats_publication_classified = pd.read_csv("data/0510-citation-pub-data-publication-classified.csv")
/var/folders/rw/s7c9z30x7fj_2gcngv0yhv6w0000gn/T/ipykernel_13307/3257668968.py:4: DtypeWarning: Columns (12,21,22,24,33,34) have mixed types. Specify dtype option on import or set low_memory=False.
  paper_stats_speaker_order = pd.read_csv("data/0510-citation-pub-data-speaker-order-absolute.csv")


In [19]:
# Merge the dataframes column-wise, avoiding duplicate columns
paper_stats = pd.concat([paper_stats_publication_classified, paper_stats_speaker_order.loc[:, ~paper_stats_speaker_order.columns.isin(paper_stats_publication_classified.columns)]], axis=1)

# Display the merged dataframe
print(paper_stats.columns)

Index(['Researcher of Interest', 'School', 'Publication Link', 'Authors',
       'Publication date', 'Journal', 'Volume', 'Pages', 'Publisher',
       'Description',
       ...
       'Citations 2019', 'Citations 2020', 'Citations 2021', 'Citations 2022',
       'Citations 2023', 'Citations 2024', 'Total Citation Count',
       'publication_type', 'Speaker Rank', 'Speaker Rank Absolute'],
      dtype='object', length=120)


In [20]:
from rapidfuzz.process import extractOne

author_authors = author_stats["Researcher Cleaned Name"].unique()
paper_authors = paper_stats["Researcher of Interest"].unique()
author_map = {}

for paper_author in paper_authors:
    author_map[paper_author] = extractOne(paper_author, author_authors)[0]

paper_stats["Researcher of Interest Mapped"] = paper_stats["Researcher of Interest"].map(author_map)


In [27]:
pub_types = paper_stats["publication_type"].unique()
print("Publication types: ", pub_types)

# Initialize a dictionary to store the publication counts for each researcher
publication_counts = {author: {pub_type: 0 for pub_type in pub_types} for author in author_map.values()}
average_rank = {author: {rank_type: 0 for rank_type in ['Speaker Rank', 'Speaker Rank Absolute']} for author in author_map.values()}
average_rank_valid_count = {author: 0 for author in author_map.values()}

# Iterate through the paper_stats dataframe and count the publication types for each mapped researcher
for _, row in paper_stats.iterrows():
    mapped_author = row["Researcher of Interest Mapped"]
    if mapped_author in publication_counts:
        publication_type = row["publication_type"]
        if publication_type in publication_counts[mapped_author]:
            publication_counts[mapped_author][publication_type] += 1
        if row['Speaker Rank'] >= 0 and row['Speaker Rank Absolute'] >= 0:
            average_rank[mapped_author]['Speaker Rank'] += row['Speaker Rank']
            average_rank[mapped_author]['Speaker Rank Absolute'] += row['Speaker Rank Absolute']
            average_rank_valid_count[mapped_author] += 1

# Calculate the average rank for each researcher
for author in average_rank:
    average_rank[author]['Speaker Rank'] /= average_rank_valid_count[author]
    average_rank[author]['Speaker Rank Absolute'] /= average_rank_valid_count[author]


# Convert the publication_counts dictionary to a DataFrame for better visualization
publication_counts_df = pd.DataFrame(publication_counts).T
publication_counts_df.columns = ['publication_count_' + col for col in publication_counts_df.columns]
print(publication_counts_df)

# Convert the average_rank dictionary to a DataFrame for better visualization
average_rank_df = pd.DataFrame(average_rank).T
average_rank_df.columns = ['average_rank_' + col for col in average_rank_df.columns]
print(average_rank_df)

Publication types:  ['journal' 'empty_journal_publisher' 'preprint' 'conference' 'other'
 'institutional' 'book']
                       publication_count_journal  \
Cai Wentong                                  130   
Nachiket Ganesh Kapre                         17   
Bo An                                         66   
Xiao Xiaokui                                  53   
Chng Eng Siong                                38   
...                                          ...   
BROWN, Michael Scott                          48   
GOH Khim Yong                                 24   
KANKANHALLI, Atreyi                           66   
HENZ Martin                                   19   
BRESSAN, Stephane                             39   

                       publication_count_empty_journal_publisher  \
Cai Wentong                                                  132   
Nachiket Ganesh Kapre                                         25   
Bo An                                                    

In [28]:
# Merge the publication_counts_df with author_stats based on the mapped researcher names
merged_df = author_stats.merge(publication_counts_df, left_on='Researcher Cleaned Name', right_index=True, how='left')
merged_df = merged_df.merge(average_rank_df, left_on='Researcher Cleaned Name', right_index=True, how='left')
# Display the merged dataframe
print(merged_df.head())

  Researcher Cleaned Name    year  month  num_sg_mean  num_sg_error  \
0          A S Madhukumar  1991.0    8.0          0.0           0.0   
1          A S Madhukumar  1992.0   11.0          0.0           0.0   
2          A S Madhukumar  1993.0    7.0          0.0           0.0   
3          A S Madhukumar  1994.0    2.0          0.0           0.0   
4          A S Madhukumar  1997.0    9.0          0.0           0.0   

   num_hk_mean  num_hk_error  num_ml_mean  num_ml_error  num_other_mean  ...  \
0          0.0           0.0          1.0           0.0             2.0  ...   
1          0.0           0.0          1.0           0.0             2.0  ...   
2          0.0           0.0          1.0           0.0             2.0  ...   
3          0.0           0.0          1.0           0.0             3.0  ...   
4          0.0           0.0          0.0           0.0             2.0  ...   

   total_camouflage_score  publication_count_journal  \
0                     0.0           

In [29]:
print(merged_df.columns)

Index(['Researcher Cleaned Name', 'year', 'month', 'num_sg_mean',
       'num_sg_error', 'num_hk_mean', 'num_hk_error', 'num_ml_mean',
       'num_ml_error', 'num_other_mean', 'num_other_error', 'teamsize_mean',
       'teamsize_error', 'prop_sg_mean', 'prop_sg_error', 'prop_hk_mean',
       'prop_hk_error', 'prop_ml_mean', 'prop_ml_error', 'prop_other_mean',
       'prop_other_error', 'total_citations_mean', 'total_citations_error',
       'Name', 'Researcher Classification', 'Left after 2014', 'PhD Location',
       'Master Location', 'Bachelor Location', 'Leaving Region', 'Join Year',
       'Join Month', 'PhD Year', 'Master Year', 'Bachelor Year', 'Department',
       'Title', 'First Year', 'Leaving Year', 'Leaving Month', 'Academic Age',
       'Pre-2014 School', 'Post Event', 'Pre-2014 Region',
       'Number of Publications', 'general_analysis',
       'cross_group_thinking_ideas', 'cross_group_thinking_score',
       'unifying_language_ideas', 'unifying_language_score',
       

In [42]:
# Iterate through the columns ending with '_ideas'
for col in merged_df.columns:
    if col.endswith('_ideas'):
        # Check if the column has any 0 values
        zero_indices = merged_df[merged_df[col] == '0'].index
        print(f"Column '{col}' has {len(zero_indices)} zero values")

        # Count the number of NaNs in the column
        nan_count = merged_df[col].isna().sum()
        print(f"Column '{col}' has {nan_count} NaN values")

        # Set the corresponding '_ideas' and '_score' columns to NaN
        for zero_index in zero_indices:
            for inner_col in merged_df.columns:
                if inner_col.endswith('_ideas'):
                    merged_df.at[zero_index, inner_col] = pd.NA
                if inner_col.endswith('_score'):
                    merged_df.at[zero_index, inner_col] = pd.NA

# Display the updated dataframe
print(merged_df.head())


Column 'cross_group_thinking_ideas' has 12945 zero values
Column 'cross_group_thinking_ideas' has 0 NaN values
Column 'unifying_language_ideas' has 0 zero values
Column 'unifying_language_ideas' has 12945 NaN values
Column 'shared_initiatives_outreach_ideas' has 0 zero values
Column 'shared_initiatives_outreach_ideas' has 12945 NaN values
Column 'importance_of_career_standing_ideas' has 0 zero values
Column 'importance_of_career_standing_ideas' has 12945 NaN values
Column 'hong_kong_emphasis_ideas' has 0 zero values
Column 'hong_kong_emphasis_ideas' has 12945 NaN values
Column 'highlighting_china_organization_ideas' has 0 zero values
Column 'highlighting_china_organization_ideas' has 12945 NaN values
  Researcher Cleaned Name    year  month  num_sg_mean  num_sg_error  \
0          A S Madhukumar  1991.0    8.0          0.0           0.0   
1          A S Madhukumar  1992.0   11.0          0.0           0.0   
2          A S Madhukumar  1993.0    7.0          0.0           0.0   
3     

In [43]:
merged_df.iloc[:, -20:].head()

,shared_initiatives_outreach_ideas,shared_initiatives_outreach_score,importance_of_career_standing_ideas,importance_of_career_standing_score,hong_kong_emphasis_ideas,hong_kong_emphasis_score,highlighting_china_organization_ideas,highlighting_china_organization_score,inverse_hong_kong_emphasis_score,total_reconciliation_score,total_camouflage_score,publication_count_journal,publication_count_empty_journal_publisher,publication_count_preprint,publication_count_conference,publication_count_other,publication_count_institutional,publication_count_book,average_rank_Speaker Rank,average_rank_Speaker Rank Absolute
0,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,NaN,NaN,NaN,101.0,31.0,167.0,20.0,0.0,6.0,0.0,0.725998,2.431677
1,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,NaN,NaN,NaN,101.0,31.0,167.0,20.0,0.0,6.0,0.0,0.725998,2.431677
2,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,NaN,NaN,NaN,101.0,31.0,167.0,20.0,0.0,6.0,0.0,0.725998,2.431677
3,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,NaN,NaN,NaN,101.0,31.0,167.0,20.0,0.0,6.0,0.0,0.725998,2.431677
4,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,NaN,NaN,NaN,101.0,31.0,167.0,20.0,0.0,6.0,0.0,0.725998,2.431677


In [44]:
merged_df.to_csv("data/0721-month-analysis-data-author-stats-merged.csv")